In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [81]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [82]:
%run -i ../../core.py

In [83]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [84]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.mean(dummy_data_tensor)



In [85]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<=torch.abs(0.01*X.size()[1]*self.w), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [86]:
# prover calculates settings, send to verifier

theory_output = torch.mean(data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<=torch.abs(0.01*X.size()[1]*self.w), self.w)
prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

Theory_output:  tensor(49.9700)
==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":10,"lookup_range":[-9594,0],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":610,"total_const_size":4,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [87]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0


Time setup: 1.6543021202087402 seconds
Theory output:  tensor(49.9700)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.96875
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[17236403609878300513, 8975289109583201277, 11033320042856086930, 2298871365537728986], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [4072474832971751158, 13621019950687641747, 8349885009801153069, 3312433242047143324]]], 'proof': '1024fd61295360336453acc7dc1ab4a5f6f2da4b8b52a0c98a7882f1f8050c5f1864291eee136142c4c6b3f7d80669f6f9fef818a8d3b8194047e4f297ff5be11421c6ee2710054745dad3dc39312f14b82c642bf95715eeb6ac85c3f36d74f22abca23d0d4ffb81b72891ac3e1b2b7e06a0a3ecb29e6ad4ded3560244333617102feee6cebb2b1db8f3c40a47ddd818956816b01cd46826f3fc20ab933f039d13cb4b43685c15544529107f8c50037d3edab3c9064396b1d3d8c2f65d23cac014b6e5fb563cac9818dc9502dbb8891b25a7596abd57536cad5a47e17b081d94005eea7371d0ca48c82fb06bff97e6e3ca5b1ec36a11408fe60ba96191783adc27d7489c55cb00b867c3f75d295179a748f28586ce76fd9b95578f729d884d921531b17e4c24468cc2653414222487813fb318d091b77717fffec67c34e8acb50c68a2c14f431b21f574d2d2b6abf13df60718926b6d2b32cf61f50c03f64f

In [88]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[17236403609878300513, 8975289109583201277, 11033320042856086930, 2298871365537728986], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [4072474832971751158, 13621019950687641747, 8349885009801153069, 3312433242047143324]]]
proof boolean:  1.0
proof result 1 : 49.96875
verified
